# Hecho de servicio por fases

In [2]:
import datetime
import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine


# Añadir conexión a la base de datos

In [2]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

8) Mostrar los tiempos de espera por cada fase del servicio: Iniciado, Con mensajero asignado, recogido en origen, Entregado en Destino, Cerrado. En que fase del servicio hay más demoras?

# Extracción de datos

In [ ]:
servicios_df = pd.read_sql("SELECT * FROM mensajeria_servicio", ryf_conn)
estados_df = pd.read_sql("SELECT * FROM mensajeria_estadosservicio", ryf_conn)

# Transformación

In [ ]:
estados_servicio = estados_df.merge(servicios_df, left_on="servicio_id", right_on="id", suffixes=('_estado', '_servicio'))
estados_servicio['duracion_fase'] = estados_servicio.groupby("servicio_id")['fecha'].diff()

# Filtrar columnas relevantes

In [ ]:
hecho_servicio_fases_df = estados_servicio[['servicio_id', 'estado_id', 'fecha', 'duracion_fase']]

# Carga al datamart

In [ ]:
hecho_servicio_fases_df.to_sql("hecho_servicio_fases", etl_conn, if_exists="replace", index=False)